In [7]:
import requests
import json
from bs4 import BeautifulSoup
from csv import writer

client_id = 'client_660b4d0cceb4cf9f989acef154caf4ab'
client_secret = 'secret_bd749d4e48d050deb1d8873b061456f2'

def retrive_properties(client_id, client_secret, pageNumber, pageSize=200, state="VIC", filename="propertiesInfo.csv"):
  scopes = ['api_listings_read']
  auth_url = 'https://auth.domain.com.au/v1/connect/token'
  url_endpoint = 'https://api.domain.com.au/v1/listings/residential/_search'

  response = requests.post(auth_url, data = {
                          'client_id':client_id,
                          'client_secret':client_secret,
                          'grant_type':'client_credentials',
                          'scope':scopes,
                          'Content-Type':'text/json'
                          })
  json_res = response.json()
  access_token=json_res['access_token']
  print("Access:", access_token)
  auth = {'Authorization':'Bearer ' + access_token}
  url = url_endpoint

  post_fields = {
        "listingType":"Rent",
        "pageSize": 200, 
        "pageNumber": 1, # 10 per page
        "propertyTypes":"",
        "minBedrooms":1,
        "minBathrooms":1,
        # "maxPrice": 400,
        "sort": {
            "sortKey": "Default"
        },

        "locations":[
          {
            "state":state,
            "region":"",
            "area":"",
            "suburb":"",
            "postCode":"",
            "includeSurroundingSuburbs":False
          }
        ]
  }

  # Request content
  respond = requests.post(url,headers=auth,json=post_fields)
  content = json.loads(respond.text)
  print("Loaded data: " + str(len(content)))

  # Get request for each property ID and extract attributes
  for i in content:
      #get details
      id = i['listing']['id']
      listing_type = i['listing']['listingType']
      r = i['listing']['propertyDetails']
      street_address=r['displayableAddress']
      suburb=r['suburb']
      postcode=r['postcode']
      area=r['area']
      latitude = r['latitude']
      longitude = r['longitude']
      property_type=r['propertyType']
      bathrooms=r['bathrooms']
      bedrooms=r['bedrooms']

      if(str(i).find('carspaces'))>0:
          carspaces=i['listing']['propertyDetails']['carspaces']
      else:
          carspaces=0

      price=i['listing']['priceDetails']['displayPrice'] #[1:4]

      info = [id, listing_type, price, property_type, area, bedrooms, bathrooms, carspaces, street_address, suburb, postcode, latitude, longitude]

      with open(filename, 'a', newline='') as g:
          thewriter = writer(g)
          thewriter.writerow(info)

retrive_properties(client_id, client_secret, 1)

Access: f3981e38d2a4cea1ce0a7f86e3648d06
Loaded data: 200
